# Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
pd.set_option('display.max_columns', None)

# Check Valid Game Functions

In [4]:
def check_1d(arr):
    return np.all(np.sort(arr) == np.arange(1,10))

def check_grids(game):
    for row_index_start in np.arange(0,7,3):
        for col_index_start in np.arange(0,7,3):
            check_grid = check_1d(game[row_index_start:row_index_start+3,col_index_start:col_index_start+3].flatten())
            if not check_grid:
                return False
    return True
    
def check_valid_sudoku(game):
    row_check = np.all(np.apply_along_axis(check_1d,axis =1 ,arr= game))
    col_check = np.all(np.apply_along_axis(check_1d,axis =0 ,arr= game))
    grid_check = check_grids(game)
    return np.all([grid_check, row_check, col_check])

def check_all_data_set(data_set):
    invalid_game_list = []
    for i,game in enumerate(data_set.reshape(-1,9,9)):
        if not check_valid_sudoku(game):
            invalid_game_list.append(i)
    if invalid_game_list:
        print(f'Invalid Games at Indices {invalid_game_list}')
        return False
    print('No Invalid Games')
    return True


# Model Functions

In [15]:
def build_model(output_activation = 'softmax', loss = 'sparse_categorical_crossentropy' , learning_rate= 0.001):
    inputs = tf.keras.Input(shape=(9, 9, 1))
    x = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(512, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(1024, 3, activation='relu', padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(9, 1, activation='relu', padding='same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512)(x)
    x = tf.keras.layers.Dense(81*9)(x)
    x = tf.keras.layers.LayerNormalization(axis=-1)(x)
    x = tf.keras.layers.Reshape((9, 9, 9))(x)
    outputs = tf.keras.layers.Activation(output_activation)(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

# Set Pathing

In [5]:
training_games_dir = 'sudoku_games'
training_games_file = 'sudoku.csv'
training_path = os.path.join(training_games_dir, training_games_file)

# Read in File

In [7]:
sudoku_df = pd.read_csv(training_path)

In [9]:
X = np.array(sudoku_df.quizzes.map(lambda x: list(map(int, x))).to_list())[0:10_000]
y = np.array(sudoku_df.solutions.map(lambda x: list(map(int, x))).to_list())[0:10_000]

# Check Dataset is Full of Valid Solutions

In [10]:
# np.set_printoptions(threshold=np.inf)
test_game = y.reshape(-1,9,9)[0].copy()
# test_game[0,0] = 6

In [11]:
check_valid_sudoku(test_game)

True

In [9]:
check_all_data_set(y)

No Invalid Games


True

# Reshape Train and Test for CNN

In [41]:
X = X.reshape(-1,9,9,1)
y = y.reshape(-1,9,9) - 1

# Create Test Train Split

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape

(8000, 9, 9, 1)

# Build Model

In [44]:
model = build_model()
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 9, 9, 1)]         0         
                                                                 
 conv2d_24 (Conv2D)          (None, 9, 9, 64)          640       
                                                                 
 batch_normalization_23 (Bat  (None, 9, 9, 64)         256       
 chNormalization)                                                
                                                                 
 conv2d_25 (Conv2D)          (None, 9, 9, 64)          36928     
                                                                 
 batch_normalization_24 (Bat  (None, 9, 9, 64)         256       
 chNormalization)                                                
                                                                 
 flatten_9 (Flatten)         (None, 5184)              0   

# Train Model

In [45]:
checkpoint_dir = os.path.join(os.getcwd(),'sudoku_solver_models')
checkpoint_path = os.path.join(checkpoint_dir,'best_sudoku_solver_test_training_cnn_test.h5')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
with tf.device('/GPU:0'):
    history = model.fit(X_train,y_train, epochs=10,verbose=2,batch_size=64,validation_data = (X_test,y_test),callbacks=[checkpoint_callback,early_stop])

Epoch 1/10


ValueError: in user code:

    File "C:\Users\MA51801\Documents\Projects\computervision\venv\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\MA51801\Documents\Projects\computervision\venv\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\MA51801\Documents\Projects\computervision\venv\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\MA51801\Documents\Projects\computervision\venv\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\MA51801\Documents\Projects\computervision\venv\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "C:\Users\MA

In [20]:
X_test[1].reshape(9,9)

array([[0, 1, 6, 9, 0, 4, 0, 0, 7],
       [0, 0, 4, 0, 3, 0, 0, 8, 0],
       [0, 0, 3, 0, 6, 1, 9, 2, 0],
       [5, 0, 9, 1, 4, 0, 8, 0, 0],
       [1, 7, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 8, 7, 0, 0, 0, 6, 5],
       [6, 0, 0, 0, 0, 2, 0, 4, 0],
       [0, 2, 0, 8, 0, 5, 3, 1, 0],
       [0, 3, 0, 0, 0, 0, 0, 0, 9]])

In [21]:
y_test[1].reshape(9,9) + 1

array([[2, 1, 6, 9, 8, 4, 5, 3, 7],
       [9, 5, 4, 2, 3, 7, 6, 8, 1],
       [7, 8, 3, 5, 6, 1, 9, 2, 4],
       [5, 6, 9, 1, 4, 3, 8, 7, 2],
       [1, 7, 2, 6, 5, 8, 4, 9, 3],
       [3, 4, 8, 7, 2, 9, 1, 6, 5],
       [6, 9, 5, 3, 1, 2, 7, 4, 8],
       [4, 2, 7, 8, 9, 5, 3, 1, 6],
       [8, 3, 1, 4, 7, 6, 2, 5, 9]])

In [22]:
sol = model.predict(X_test[1].reshape(1, 9, 9, 1)).argmax(-1).squeeze()+1

1/1 [==============================] - 0s 49ms/step


In [23]:
sol

array([[2, 1, 6, 9, 8, 4, 5, 3, 7],
       [7, 5, 4, 2, 3, 7, 6, 8, 1],
       [7, 8, 3, 5, 6, 1, 9, 2, 4],
       [5, 6, 9, 1, 4, 3, 8, 7, 2],
       [1, 7, 2, 6, 5, 8, 4, 9, 3],
       [3, 4, 8, 7, 2, 9, 1, 6, 5],
       [6, 9, 1, 3, 9, 2, 7, 4, 8],
       [4, 2, 7, 8, 9, 5, 3, 1, 6],
       [8, 3, 5, 4, 1, 6, 2, 5, 9]], dtype=int64)

In [24]:
check_valid_sudoku(sol)

False

In [49]:
X_test[1]

array([[[0],
        [2],
        [5],
        [4],
        [6],
        [3],
        [0],
        [0],
        [0]],

       [[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1],
        [0]],

       [[0],
        [0],
        [7],
        [0],
        [0],
        [0],
        [6],
        [0],
        [2]],

       [[0],
        [3],
        [1],
        [0],
        [0],
        [7],
        [0],
        [0],
        [4]],

       [[9],
        [8],
        [0],
        [0],
        [5],
        [4],
        [1],
        [2],
        [0]],

       [[0],
        [0],
        [0],
        [9],
        [0],
        [0],
        [0],
        [8],
        [3]],

       [[4],
        [0],
        [0],
        [1],
        [2],
        [0],
        [3],
        [0],
        [6]],

       [[3],
        [0],
        [0],
        [0],
        [0],
        [0],
        [8],
        [5],
        [0]],

       [[7],
        [0],
        [0],
        [

# Imports